In [786]:
from collections import OrderedDict
from conllu import parse
from enum import Enum
import numpy as np
import json
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
import tokenize_uk
import pymorphy2
import stanza
import spacy
from keras.models import Sequential
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import SimpleRNN, LSTM
from keras.layers.convolutional import Conv1D
from keras.layers.pooling import MaxPooling1D
from keras.layers.core import Dense
from keras.layers.wrappers import TimeDistributed
from keras.layers import Dropout, Masking
from keras.utils import to_categorical
from conllu import parse
from gensim.models.word2vec import Word2Vec, LineSentence
from gensim.models import KeyedVectors
from enum import Enum
from collections import OrderedDict
import progressbar
import tensorflow as tf
import random as python_random

In [3]:
PATH = '../../../../UD_Ukrainian-IU'

with open(PATH + '/uk_iu-ud-train.conllu') as f:
    train_data = f.read()
    
with open(PATH + '/uk_iu-ud-dev.conllu') as f:
    test_data = f.read()

train_trees = parse(train_data)
test_trees = parse(test_data)

In [4]:
nlp_uk = spacy.load('/Users/lissm/Work/Dev/NLP/corpora/vectors/ubercorpus_lowercased_tokenized')

In [66]:
# class Actions(str, Enum):
#     SHIFT = 'shift'
#     REDUCE = 'reduce'
#     RIGHT = 'right'
#     LEFT = 'left'
    
class Actions(int, Enum):
    SHIFT = 0
    REDUCE = 1
    RIGHT = 2
    LEFT = 3


ROOT = OrderedDict([('id', 0), ('form', 'ROOT'), ('lemma', 'ROOT'), ('upostag', 'ROOT'),
                    ('xpostag', None), ('feats',
                                        None), ('head', None),  ('deprel', None),
                    ('deps', None), ('misc', None)])


# def dep_parse(tree, clf, vectorizer, feature_extractor):
#     stack, queue, relations = [ROOT], tree[:], []

#     while queue or stack:
#         if stack and not queue:
#             stack.pop()
#         else:
#             features = feature_extractor(stack, queue, relations)
#             action = clf.predict(vectorizer.transform([features]))[0]

#             if action == Actions.SHIFT:
#                 stack.append(queue.pop(0))
#             elif action == Actions.REDUCE:
#                 stack.pop()
#             elif action == Actions.LEFT:
#                 deprel = stack[-1]['deprel'] or predict_deprel(
#                     stack[-1], queue[0], stack + queue)
#                 rel = (stack[-1]['id'], queue[0]['id'], deprel)
#                 relations.append(rel)
#                 stack.pop()
#             elif action == Actions.RIGHT:
#                 deprel = queue[0]['deprel'] or predict_deprel(
#                     queue[0], stack[-1], stack + queue)
#                 rel = (queue[0]['id'], stack[-1]['id'], deprel)
#                 relations.append(rel)
#                 stack.append(queue.pop(0))

#     return sorted(relations)

In [6]:
""" Data & reporting utils """


def calculate_as(trees, clf, vect, feature_extractor):
    total, tpu, tpl, full_match = 0, 0, 0, 0
    golden_u, golden_l = None, None
    for tree in trees:
        tree = [t for t in tree if type(t['id']) == int]
        golden_all = [(node['id'], node['head'], node['deprel'])
                      for node in tree]
        golden_u = [(x, y) for x, y, _ in golden_all]

        predicted_all = dep_parse(tree, clf, vect, feature_extractor)
        predicted_u = [(x, y) for x, y, _ in predicted_all]

        total += len(tree)
        tpu += len(set(golden_u).intersection(set(predicted_u)))
        tpl += len(set(golden_all).intersection(set(predicted_all)))

        if set(golden_all) == set(predicted_all):
            full_match += 1

    print('== Attachment score report ==')
    print('Total: ', total)
    print('Match unlabeled: ', tpu)
#     print('Match labeled: ', tpl)
    print('UAS: ', round(tpu/total, 2))
#     print('LAS: ', round(tpl/total, 2))
    print("Full match:", round(full_match/len(trees), 2))


def get_lrc_classifier():
    pipe = Pipeline([
        ('dict_vect', DictVectorizer()),
        ('lrc', LogisticRegression(random_state=42, multi_class='multinomial',
                                   max_iter=100, solver='sag', n_jobs=20))])

    return pipe


def print_result(train_trees, test_trees, clf, feature_extractor):
    train_feat, train_lab = get_data(train_trees, feature_extractor)
    test_feat, test_lab = get_data(test_trees, feature_extractor)

    clf.fit(train_feat, train_lab)
    print(classification_report(test_lab, clf.predict(test_feat)))
    calculate_as(test_trees, clf['lrc'], clf['dict_vect'], feature_extractor)

In [773]:
def get_w2v_model(trees):
    words = []
    for tree in trees:
        words.append([x['lemma'] for x in tree])
    model = Word2Vec(
        words, size=300, min_count=1, workers=6,
        window=5, alpha=0.030, negative=7, iter=5
    )

    return model

def oracle(stack, top_queue, relations):
    """
    Make a decision on the right action to do.
    """
    top_stack = stack[-1]
    # check if both stack and queue are non-empty
    if top_stack and not top_queue:
        return Actions.REDUCE
    # check if there are any clear dependencies
    elif top_queue['head'] == top_stack['id']:
        return Actions.RIGHT
    elif top_stack['head'] == top_queue['id']:
        return Actions.LEFT
    # check if we can reduce the top of the stack
    elif top_stack['id'] in [i[0] for i in relations] and \
        (top_queue['head'] < top_stack['id'] or
         [s for s in stack if s['head'] == top_queue['id']]):
        return Actions.REDUCE
    # default option
    else:
        return Actions.SHIFT


def get_data_for_tree(tree):
    res = []
    stack, buffer, relations = [ROOT], tree[:], []

    while buffer or stack:
        action = oracle(stack if len(stack) > 0 else None,
                        buffer[0] if len(buffer) > 0 else None,
                        relations)
        if len(stack) and len(buffer):
            res.append((stack[-1]['lemma'], buffer[0]['lemma'], action.value))
        if action == Actions.SHIFT:
            stack.append(buffer.pop(0))
        elif action == Actions.REDUCE:
            stack.pop()
        elif action == Actions.LEFT:
            rel = (stack[-1]['id'], buffer[0]['id'])
            relations.append(rel)
            stack.pop()
        elif action == Actions.RIGHT:
            rel = (buffer[0]['id'], stack[-1]['id'])
            relations.append(rel)
            stack.append(buffer.pop(0))

    return res


def get_data(trees, v2w_model):
    vectors, labels = [], []
    for tree in trees:
        t_vectors, t_labels = [], []
        tree_data = get_data_for_tree(
            [t for t in tree if type(t['id']) == int])
        for stack, buff, label in tree_data:
            try:
                stack_vect = v2w_model.wv[stack]
                buff_vect = v2w_model.wv[buff]
                vectors.append(np.concatenate((stack_vect, buff_vect)))
                labels.append(label)
            except Exception as e:
                pass

    return np.moveaxis(np.dstack(vectors), -1, 0), np.dstack(labels)


def get_predicted_label(labels):
    l = labels[0][0]
    return np.argmax(l)


def get_target_data(pairs, v2w_model):
    vectors = []
    for stack, buffer in pairs:
        try:
            stack_vect = v2w_model.wv[stack]
            buff_vect = v2w_model.wv[buff]
            vectors.append(np.concatenate((stack_vect, buff_vect)))
        except:
            pass

    return np.dstack(vectors)

In [820]:
def make_model(n_vocab, X):
    model = Sequential()
    model.add(LSTM(4, return_sequences=True, activation='relu', input_shape=(X.shape[1], X.shape[2])))
    model.add(LSTM(512, return_sequences=True, activation='relu'))
#     model.add(Dropout(0.5))
    model.add(Dense(4, activation='softmax'))
    model.compile('adam', 'categorical_crossentropy', metrics=['accuracy'])
    print(model.summary())

    return model


def to_cat(y_int, vocab_size):
    return to_categorical(y_int, num_classes=vocab_size)


def train_model(model, epochs, x, y, n_classes):
    model.fit(x, y, epochs=epochs, verbose=1)

    
def predict(model, x):
    return model.predict(x)

In [787]:
np.random.seed(42)
python_random.seed(42)
tf.random.set_seed(42)

In [821]:
v2w_model = get_w2v_model(train_trees)

2020-05-23 22:48:14 WARNING: under 10 jobs per worker: consider setting a smaller `batch_words' for smoother alpha decay


In [817]:
train_x, train_y = get_data(train_trees, v2w_model)
n_vocab = len(train_x)
n_classes = 4

In [822]:
model = make_model(n_vocab, train_x)

Model: "sequential_53"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_70 (LSTM)               (None, 1, 4)              9680      
_________________________________________________________________
lstm_71 (LSTM)               (None, 1, 512)            1058816   
_________________________________________________________________
dense_29 (Dense)             (None, 1, 4)              2052      
Total params: 1,070,548
Trainable params: 1,070,548
Non-trainable params: 0
_________________________________________________________________
None


In [823]:
y = np.moveaxis(train_y, -1, 0)
y = np.array([to_cat(x, n_classes) for x in y])
train_model(model, 20, train_x, y, n_classes)

Epoch 1/20
160186/160186 [==============================] - 90s 564us/step - loss: 1.1018 - accuracy: 0.5126
Epoch 2/20
160186/160186 [==============================] - 91s 566us/step - loss: 1.0641 - accuracy: 0.5495
Epoch 3/20
160186/160186 [==============================] - 91s 568us/step - loss: 1.0426 - accuracy: 0.5604
Epoch 4/20
160186/160186 [==============================] - 90s 560us/step - loss: 1.0302 - accuracy: 0.5646
Epoch 5/20
160186/160186 [==============================] - 91s 568us/step - loss: 1.0250 - accuracy: 0.5664
Epoch 6/20
160186/160186 [==============================] - 91s 571us/step - loss: 1.0220 - accuracy: 0.5676
Epoch 7/20
160186/160186 [==============================] - 92s 575us/step - loss: 1.0211 - accuracy: 0.5675
Epoch 8/20
160186/160186 [==============================] - 91s 570us/step - loss: 1.0166 - accuracy: 0.5698
Epoch 9/20
160186/160186 [==============================] - 91s 569us/step - loss: 1.0151 - accuracy: 0.5717
Epoch 10/20
160186/